In [1]:
import os
os.environ['GROQ_API_KEY']= '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'
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = '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'

Loading Documents

In [61]:
#loading documents

#'https://delong.typepad.com/files/muquaddimah.pdf'
import bs4
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader(
    web_paths=(
  #    'https://isetsf.rnu.tn/',
      'https://iort.tn/SiteIORT/PAGE_ActualiteUn/tBAAAMrXBIxVeWtYZmd5Z1JaCQA?WD_ACTION_=MENU&ID=M16#M49',
      'https://www.iort.tn/SiteIORT/PAGE_Constitution/tBAAAHP9BIxVeWtYZmd5Z1JaCQA#M49',
      'https://www.iort.tn/SiteIORT/PAGE_Constitution/tBAAAFmOBYxVeWtYZmd5Z1JaCQA#M49',
      ),
    # bs_kwargs=dict(
     #   parse_only=bs4.SoupStrainer(
      #      class_=("post-content", "post-title", "post-header")
      #  )
    #),
)
blog_docs = loader.load()
print(blog_docs)


[Document(metadata={'source': 'https://iort.tn/SiteIORT/PAGE_ActualiteUn/tBAAAMrXBIxVeWtYZmd5Z1JaCQA?WD_ACTION_=MENU&ID=M16#M49', 'title': 'المطبعة الرسمية للجمهورية التونسية', 'language': 'ar'}, page_content='\n\n\nالمطبعة الرسمية للجمهورية التونسية\n\n\n\n\n\n\n\n\n\n\n\nالجمهورية التونسيةرئاسة الحكومةدستور الجمهورية التونسيةتحميل\n\nالمطبعة الرسمية للجمهورية التونسيةwww.iort.gov.tn\nالرائد الرسمي للقوانين و الأوامر و القراراترائد عدد   : 141 بتاريخ  2024/11/22\nالرائد الرسمي للإعلانات القانونية والشرعية و العدليةرائد عدد   : 136 بتاريخ  2024/11/22\nالرائد الرسمي لإعلانات المحكمة العقاريةرائد عدد   : 091 بتاريخ  \nالجريدة الرسمية للجماعات المحليةالجريدة عدد   : 087 بتاريخ  2024/11/22www.jocl.tn\nاقتناء عن بعدLien\nالمنتجاتLien\nدستور الجمهورية التونسية\nقائمة الخدماتتقديم المطبعة الرسميةبطاقة هويةالنشأة و التطورالمهامبيانات و أرقامطلبات العروضالحق في النفاذ إلى المعلومةمتحفالمكتبة الافتراضية\xa0دستور الجمهورية التونسيةالرائد الرسمي للقوانين و الأوامرو القرارات\xa0أخر عدد للرائد الرسم

splitting the text into subelements

In [62]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300, 
    chunk_overlap=50)

# Make splits
splits = text_splitter.split_documents(blog_docs)


print("Number of documents:", len(splits))
# print("Example document:", splits[0] if splits else "No documents found")

Number of documents: 117


vector store

In [63]:
from langchain_community.vectorstores import Chroma
from groqEmbeddings import GroqEmbeddings
# Use your GroqEmbeddings class
vectorstore = Chroma.from_documents(
    documents=splits, 
    embedding=GroqEmbeddings(api_key="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")
)

retriever = vectorstore.as_retriever()

In [65]:
docs = retriever.get_relevant_documents("explain what the president of the tunsian country is able todo")
docs

[Document(metadata={'description': "L’Institut Supérieur des Etudes Technologiques de Sfax est un établissement public doté de la personnalité civile et de l’autonomie financière. Relevant au ministère de l'Enseignement Supérieur et de la Recherche Scientifique. Sa création est décrétée par la loi n°92-51 du 18 mai 1992.\r\n ", 'language': 'No language found.', 'source': 'https://isetsf.rnu.tn/', 'title': 'ISETSF - Institut Supérieur des Etudes Technologiques de Sfax'}, page_content='Visite virtuelle                                \n\n\n\n\n\r\n                                    Vidéo                                \n\n\n\n\n\n\n\n\r\n                        \r\n                    Départements'),
 Document(metadata={'language': 'ar', 'source': 'https://iort.tn/SiteIORT/PAGE_ActualiteUn/tBAAAMrXBIxVeWtYZmd5Z1JaCQA?WD_ACTION_=MENU&ID=M16#M49', 'title': 'المطبعة الرسمية للجمهورية التونسية'}, page_content='القرارات\xa0أخر عدد للرائد الرسميالبحث عن الرائد الرسميالبحث عن النصبكلمةبحث معمقا

In [66]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [67]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer the question based only on the following context:  {context}",
        ),
        ("human", "{input}"),
    ]
)

chain = prompt | llm
chain.invoke(
    {
        "context": docs,
        "input": "explain what the president of the tunsian country is able todo",
    }
)

AIMessage(content='Based on the provided context, specifically the document from the Tunisian Official Gazette (المطبعة الرسمية للجمهورية التونسية), the powers and responsibilities of the President of Tunisia are not explicitly stated. However, I can provide some general information about the role of the President in Tunisia.\n\nAccording to the Tunisian Constitution, the President is the head of state and has several key powers and responsibilities, including:\n\n1. Representing the state and the nation\n2. Ensuring the continuity of the state and its institutions\n3. Appointing the Prime Minister and other government officials\n4. Signing laws and decrees\n5. Serving as the commander-in-chief of the armed forces\n6. Representing Tunisia internationally\n7. Granting pardons and reprieves\n8. Dissolving the parliament and calling for new elections\n\nHowever, it\'s worth noting that the provided context does not specifically mention these powers, and the actual powers and responsibilit